In [1]:
import os
import wikipedia
import tiktoken
from dotenv import load_dotenv
from openai import OpenAI
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate


In [2]:
load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

openai_client = OpenAI(api_key=OPENAI_API_KEY)

In [3]:
embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

In [4]:
LLM_CONFIG = {
    'model': 'gpt-4o-mini',
    'api_key': OPENAI_API_KEY
}

In [5]:
# Pinecone
vectorstore = PineconeVectorStore(
    index_name='langchain-fastapi', embedding=embeddings
)

index_name = 'langchain-fastapi'

namespace = 'Search Wikipedia'

In [6]:
# Initialize Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

# Connect to your pinecone index
pinecone_index = pc.Index('langchain-fastapi')

In [7]:
retriever = vectorstore.as_retriever(
    search_type='similarity',
    search_kwargs={'k': 2}
)

retriever.invoke("What is the city named after trees")

retriever

VectorStoreRetriever(tags=['PineconeVectorStore', 'OpenAIEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x0000018917AB8830>, search_kwargs={'k': 2})

In [8]:
llm = ChatOpenAI(**LLM_CONFIG)

In [10]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000002C3D9FDBF20>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000002C3DB0A4260>, root_client=<openai.OpenAI object at 0x000002C3D9E29850>, root_async_client=<openai.AsyncOpenAI object at 0x000002C3D9ED7140>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [10]:
message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""
prompt = ChatPromptTemplate.from_messages([('human', message)])

In [11]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    modified=lambda x: x['num'] * 123121,
)

runnable.invoke({'num': 31})

{'passed': {'num': 31}, 'modified': 3816751}

In [12]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

In [13]:
chain = {'context': retriever, 'question': RunnablePassthrough()} | prompt | llm | parser

In [14]:
chain.invoke('what type of food is best known in Austin Texas')

'The provided context does not contain any information about the type of food best known in Austin, Texas.'